In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
%matplotlib notebook
from Reinforcement_Learning import *
from Maze import *
from DrawMaze import *
import cv2
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm

In [2]:
PATH = 0 #道
WALL = 1 #壁
STR = 'S' #スタート
GOL = 'G' #ゴール
# mazeRead, mazeWrite, mazeDataFile = False, True, 'input/maze.csv'
mazeRead, mazeWrite, mazeDataFile = False, True, 'input/maze.csv'
if mazeRead:
    #迷路読み込み
    mazeData = []
    with open(mazeDataFile) as f:
        reader = csv.reader(f)
        for row in reader:
            mazeData.append(row)
    maze = Maze(mazeData, 1)
else:
    #迷路生成
    mWidth, mHeight = 13, 13
    maze = Maze([mWidth, mHeight], 0)
    maze.set_out_wall()
    maze.set_inner_wall_botaoshi()
    maze.set_start_goal()
    
maze.print_maze()
#迷路保存
if mazeWrite:
    maze.export_csv(mazeDataFile)
mWidth, mHeight = maze.get_width_height()
# 迷路データ maze.maze[x][y]

＃＃＃＃＃＃＃＃＃＃＃＃＃
＃Ｓ　　　　　　　　　　＃
＃　＃＃＃＃＃＃＃＃＃　＃
＃　　　　　　　＃　＃　＃
＃　＃＃＃＃＃＃＃　＃　＃
＃　　　　　　　　　＃　＃
＃＃＃　＃　＃＃＃＃＃　＃
＃　　　＃　　　　　＃　＃
＃　＃＃＃＃＃＃＃＃＃　＃
＃　　　　　　　＃　＃　＃
＃＃＃　＃＃＃　＃　＃＃＃
＃　　　　　＃　　　　Ｇ＃
＃＃＃＃＃＃＃＃＃＃＃＃＃


In [3]:
# 強化学習
# q値更新
learningRate = 0.1 #学習係数α
discountRate = 0.9 #割引率γ
qUpdate = QLearning(learningRate, discountRate) # Q学習

#行動選択方法
ep = 0.3 #ε
actionSelect = EpGreedy(ep) #ε-greedy 

learning = ReinforcementLearning(mWidth, mHeight, qUpdate, actionSelect)

In [4]:
# 学習データの読み込み、書き出し
qDataRead, qDataWrite, qMainFile, qDataFile = False, True, 'output/q_main.csv', 'output/q_data.csv'

if qDataRead:
    startAction, finAction, resetAction, sx, sy = learning.import_q_data(qMainFile, qDataFile, 300)

In [5]:
#壁でないか確認
def is_not_wall(x, y):
    if maze.maze[x][y] == WALL:
        return False
    else:
        return True

In [6]:
# 迷路描画機能
imgW = 300
imgH = 300
draw = DrawMaze(imgW, imgH, maze.maze)

In [7]:
def rand_sx_sy():
    while True:
        sx, sy = random.randrange(mWidth), random.randrange(mHeight)
        if maze.maze[sx][sy] != WALL:
            return sx, sy

In [8]:
# 学習初期化
str_x, str_y = maze.get_start() #TODO Start位置の取得関数
gol_x, gol_y = maze.get_goal() #TODO Goal位置の取得関数
gol_reward = 30 #成功報酬
try:
    startAction
except NameError:
    startAction = 0 # 開始行動数
    finAction = startAction + 50000 # 終了行動数
    resetAction = 0
#     sx, sy = str_x, str_y
    sx, sy = rand_sx_sy()
    
randAction = 25000
maxResetAction = (mWidth + mHeight) * 30 # リセットになる行動回数

In [9]:
for t in tqdm(range(startAction, finAction)):
    # 次の行動を決定
    while True:
        nx, ny, action = learning.get_next_state(sx, sy)
        # 今回は次の行動が壁の時はキャンセルする
        if is_not_wall(nx, ny):
            break
    
    # Q値更新
    sq = learning.get_q_val(sx, sy, action)
    na, nq = learning.get_q_max_val(nx, ny)
    reward = gol_reward if (nx == gol_x and ny == gol_y) else 0
    p = [sq, nq, reward]
    learning.update_q_val(sx, sy, action, p)
    
    # 迷路描画
    draw.drawMaze(learning, gol_reward, sx, sy, nx, ny, t)
    sx, sy = nx, ny
    
    resetAction = resetAction + 1
    # リセット回数に達している、またはゴールならばスタートに戻る
    if (resetAction >= maxResetAction) or (sx == gol_x and sy == gol_y):
        resetAction = 0
        if t < randAction:
            sx, sy = rand_sx_sy()
        else:
            sx, sy = str_x, str_y   
        
# 追加学習の準備
startAction = finAction

100%|██████████| 50000/50000 [03:02<00:00, 274.18it/s]


In [10]:
finAction = finAction + 5000

In [11]:
#結果出力(csv)
# csvの書き出し
if qDataWrite:
    learning.export_q_data(qMainFile, qDataFile, startAction, resetAction, sx, sy)

['x', 'y', <Action.UP: 1>, <Action.DOWN: 2>, <Action.RIGHT: 8>, <Action.LEFT: 4>, <Action.MAX: 64>]


In [12]:
#結果出力(動画)
# encoder(for mp4)
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# output file name, encoder, fps, size(fit to image size)
video = cv2.VideoWriter('video.mp4',fourcc, 120.0, (imgW, imgH))
if not video.isOpened():
    print("can't be opened")
    exit()
    
for i in tqdm(range(0, startAction)):
    # hoge0000.png, hoge0001.png,..., hoge0090.png
    img = cv2.imread('output/images/img_' + str(i)+ '.jpg')

    # can't read image, escape
    if img is None:
        print("can't read")
        break

    # add
    video.write(img)
#     print("read image " + str(i))

video.release()
print('written')

  0%|          | 176/50000 [00:00<00:28, 1753.23it/s]

read image 0
read image 1
read image 2
read image 3
read image 4
read image 5
read image 6
read image 7
read image 8
read image 9
read image 10
read image 11
read image 12
read image 13
read image 14
read image 15
read image 16
read image 17
read image 18
read image 19
read image 20
read image 21
read image 22
read image 23
read image 24
read image 25
read image 26
read image 27
read image 28
read image 29
read image 30
read image 31
read image 32
read image 33
read image 34
read image 35
read image 36
read image 37
read image 38
read image 39
read image 40
read image 41
read image 42
read image 43
read image 44
read image 45
read image 46
read image 47
read image 48
read image 49
read image 50
read image 51
read image 52
read image 53
read image 54
read image 55
read image 56
read image 57
read image 58
read image 59
read image 60
read image 61
read image 62
read image 63
read image 64
read image 65
read image 66
read image 67
read image 68
read image 69
read image 70
read image 71
re

  1%|          | 387/50000 [00:00<00:26, 1846.76it/s]

read image 354
read image 355
read image 356
read image 357
read image 358
read image 359
read image 360
read image 361
read image 362
read image 363
read image 364
read image 365
read image 366
read image 367
read image 368
read image 369
read image 370
read image 371
read image 372
read image 373
read image 374
read image 375
read image 376
read image 377
read image 378
read image 379
read image 380
read image 381
read image 382
read image 383
read image 384
read image 385
read image 386


  1%|          | 596/50000 [00:00<00:25, 1912.37it/s]

...

read image 49418
read image 49419
read image 49420
read image 49421
read image 49422
read image 49423
read image 49424
read image 49425
read image 49426
read image 49427
read image 49428
read image 49429
read image 49430
read image 49431
read image 49432
read image 49433
read image 49434
read image 49435
read image 49436
read image 49437
read image 49438
read image 49439
read image 49440
read image 49441
read image 49442
read image 49443
read image 49444
read image 49445
read image 49446
read image 49447
read image 49448
read image 49449
read image 49450
read image 49451
read image 49452
read image 49453
read image 49454
read image 49455
read image 49456
read image 49457
read image 49458
read image 49459
read image 49460
read image 49461
read image 49462
read image 49463
read image 49464
read image 49465
read image 49466
read image 49467
read image 49468
read image 49469
read image 49470
read image 49471
read image 49472
read image 49473
read image 49474
read image 49475
read image 494

100%|█████████▉| 49797/50000 [00:23<00:00, 1963.60it/s]

read image 49783
read image 49784
read image 49785
read image 49786
read image 49787
read image 49788
read image 49789
read image 49790
read image 49791
read image 49792
read image 49793
read image 49794
read image 49795
read image 49796
read image 49797
read image 49798
read image 49799
read image 49800
read image 49801
read image 49802
read image 49803
read image 49804
read image 49805
read image 49806
read image 49807
read image 49808
read image 49809
read image 49810
read image 49811
read image 49812
read image 49813
read image 49814
read image 49815


100%|██████████| 50000/50000 [00:23<00:00, 2085.15it/s]

read image 49816
read image 49817
read image 49818
read image 49819
read image 49820
read image 49821
read image 49822
read image 49823
read image 49824
read image 49825
read image 49826
read image 49827
read image 49828
read image 49829
read image 49830
read image 49831
read image 49832
read image 49833
read image 49834
read image 49835
read image 49836
read image 49837
read image 49838
read image 49839
read image 49840
read image 49841
read image 49842
read image 49843
read image 49844
read image 49845
read image 49846
read image 49847
read image 49848
read image 49849
read image 49850
read image 49851
read image 49852
read image 49853
read image 49854
read image 49855
read image 49856
read image 49857
read image 49858
read image 49859
read image 49860
read image 49861
read image 49862
read image 49863
read image 49864
read image 49865
read image 49866
read image 49867
read image 49868
read image 49869
read image 49870
read image 49871
read image 49872
read image 49873
read image 498